In [2]:
import dotenv
import requests
import pandas as pd
import json
import os

df_esempio = pd.read_csv('diabetic_data_filtered_250.csv')
esempio_csv = df_esempio.head(30).to_string()
colonne_originali = ", ".join(df_esempio.columns)

print(colonne_originali)

encounter_id, patient_nbr, race, gender, age, weight, admission_type_id, discharge_disposition_id, admission_source_id, time_in_hospital, payer_code, medical_specialty, num_lab_procedures, num_procedures, num_medications, number_outpatient, number_emergency, number_inpatient, diag_1, diag_2, diag_3, number_diagnoses, max_glu_serum, A1Cresult, metformin, repaglinide, nateglinide, chlorpropamide, glimepiride, acetohexamide, glipizide, glyburide, tolbutamide, pioglitazone, rosiglitazone, acarbose, miglitol, troglitazone, tolazamide, examide, citoglipton, insulin, glyburide-metformin, glipizide-metformin, glimepiride-pioglitazone, metformin-rosiglitazone, metformin-pioglitazone, change, diabetesMed, readmitted


# Requests OLLAMA

In [8]:
from io import StringIO

OLLAMA_ENDPOINT = "http://localhost:11434/api/generate"

prompt = f"""Sei un generatore di dati sintetici per un ospedale. I dati riguardano pazienti diabetici.

QUESTO È L'ESEMPIO ESATTO DEL FORMATO DA SEGUIRE:

{esempio_csv}

REQUISITI FONDAMENTALI:
1. Genera ESCLUSIVAMENTE un file CSV con 20 nuove righe
2. Il CSV DEVE contenere ESATTAMENTE queste colonne (TUTTE OBBLIGATORIE), nello stesso ordine:
{colonne_originali}
3. TUTTE le celle devono avere un valore (OBBLIGATORIAMENTE)

3. REGOLE CRITICHE:
- NON aggiungere colonne extra
- NON omettere nessuna colonna
- NON aggiungere testo o commenti
- NON utilizzare punti e virgola (;) come separatore, usa la virgola (,)
- TUTTE le celle devono avere un valore
- Mantieni gli stessi tipi di dati per ogni colonna come nell'esempio
- Mantieni gli stessi formati dei valori come nell'esempio

RISPONDI ESCLUSIVAMENTE CON LE RIGHE CSV, NIENT'ALTRO
"""


modello = "llama3.2:latest"
payload = {
    "model": modello,
    "prompt": prompt,
    "stream": False
}

response = requests.post(OLLAMA_ENDPOINT, json=payload)

if response.status_code == 200:
    result = response.json()['response']
    print(result)


    csv_lines = [line for line in result.split('\n') if line.strip() and not line.startswith('*') and not line.startswith('#')]

    csv_content = '\n'.join(csv_lines)

    try:
        df = pd.read_csv(StringIO(csv_content))

        if not all(col in df.columns for col in df_esempio.columns):
            print("ERRORE: Colonne mancanti nella risposta")
            missing_cols = set(df_esempio.columns) - set(df.columns)
            print(f"Colonne mancanti: {missing_cols}")
        else:
            df = df[df_esempio.columns]

            # Salva il file
            output_path = 'ClassicPrompt/llama3.2_synthetic_diabetes_data.csv'
            os.makedirs('StoryTellingPrompt', exist_ok=True)
            df.to_csv(output_path, index=False)
            print(f"File CSV salvato correttamente in: {output_path}")

    except Exception as e:
        print(f"Errore nel processing della risposta: {e}")
        print("Risposta ricevuta:")
        print(result)
else:
    print(f"Errore nella richiesta: {response.status_code}")
    print(response.text)


encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
1,12345,Caucasian,Male,67,150.0,2,Patient Discharge,1,Acute Coronary Syndrome,Cardiology,5,2,0,0,0,1,0,85.00,6.8,3,7,9,0,0,0
2,67890,Black,Female,62,165.0,3,Patient Admission,2,Hypertension,Nephrology,10,4,1,1,1,1,1,80.00,5.6,2,6,8,0,0,0
3,34567,American Indian,Male,78,180.0,4,Patient Discharge,3,Copd,Pulmonology,11,5,0,0,0,1,0,90.

# Requests GROQ

In [10]:
from io import StringIO
import os
import requests
import pandas as pd
import dotenv

dotenv.load_dotenv()
GROQ_API_KEY = os.getenv('MISTRAL_API_KEY')
GROQ_ENDPOINT = "https://api.groq.com/openai/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {GROQ_API_KEY}",
    "Content-Type": "application/json"
}

prompt = f"""Sei un generatore di dati sintetici per un ospedale. I dati riguardano pazienti diabetici.

QUESTO È L'ESEMPIO ESATTO DEL FORMATO DA SEGUIRE:

{esempio_csv}

REQUISITI FONDAMENTALI:
1. Genera ESCLUSIVAMENTE un file CSV con 20 nuove righe
2. Il CSV DEVE contenere ESATTAMENTE queste colonne (TUTTE OBBLIGATORIE), nello stesso ordine:
{colonne_originali}
3. TUTTE le celle devono avere un valore (OBBLIGATORIAMENTE), NON POSSONO ESSERCI CELLE VUOTE

3. REGOLE CRITICHE:
- NON aggiungere colonne extra
- NON omettere nessuna colonna
- NON aggiungere testo o commenti
- NON utilizzare punti e virgola (;) come separatore, usa la virgola (,)
- TUTTE le celle devono avere un valore, non possono essere vuote o null
- Mantieni gli stessi tipi di dati per ogni colonna come nell'esempio
- Mantieni gli stessi formati dei valori come nell'esempio

RISPONDI ESCLUSIVAMENTE CON LE RIGHE CSV, NIENT'ALTRO
ATTENZIONE: Se anche una sola cella è vuota o contiene '<null>', il file sarà SCARTATO. Genera dati COMPLETI, coerenti, senza valori nulli.
"""

payload = {
    "model": "llama-3.1-8b-instant",
    "messages": [
        {
            "role": "user",
            "content": prompt
        }
    ],
    "temperature": 0.4,
    "max_tokens": 12000
}

# Effettua la richiesta
response = requests.post(GROQ_ENDPOINT, headers=headers, json=payload)

if response.status_code == 200:
    result = response.json()['choices'][0]['message']['content']
    print(result)

    csv_lines = [line for line in result.split('\n') if line.strip() and not line.startswith('*') and not line.startswith('#')]

    csv_content = '\n'.join(csv_lines)

    try:
        # Converti in DataFrame
        df = pd.read_csv(StringIO(csv_content))

        if not all(col in df.columns for col in df_esempio.columns):
            print("ERRORE: Colonne mancanti nella risposta")
            missing_cols = set(df_esempio.columns) - set(df.columns)
            print(f"Colonne mancanti: {missing_cols}")
        else:
            df = df[df_esempio.columns]

            # Salva il file
            output_path = 'ClassicPrompt/llama-3.1-8b-instant_synthetic_diabetes_data.csv'
            os.makedirs('ClassicPrompt', exist_ok=True)
            df.to_csv(output_path, index=False)
            print(f"File CSV salvato correttamente in: {output_path}")

    except Exception as e:
        print(f"Errore nel processing della risposta: {e}")
        print("Risposta ricevuta:")
        print(result)
else:
    print(f"Errore nella richiesta: {response.status_code}")
    print(response.text)

1,  323911,  55629189, Caucasian, Female, [10-20),  250.01,  1,  1,  7,  3,  Pediatrics-Endocrinology,  41,  0,  1,  0,  0,  0,  250.83,  7.5,  7.3,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  

# Request MISTRAL

In [35]:
from io import StringIO
import os
import requests
import pandas as pd
import dotenv

dotenv.load_dotenv()
MISTRAL_API_KEY = os.getenv('MISTRAL_API_KEY')
MISTRAL_ENDPOINT = "https://api.mistral.ai/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {MISTRAL_API_KEY}",
    "Content-Type": "application/json"
}

prompt = f"""Sei un generatore di dati sintetici per un ospedale. I dati riguardano pazienti diabetici.

QUESTO È L'ESEMPIO ESATTO DEL FORMATO DA SEGUIRE:

{esempio_csv}

REQUISITI FONDAMENTALI:
1. Genera ESCLUSIVAMENTE un file CSV con 20 nuove righe
2. Il CSV DEVE contenere ESATTAMENTE queste colonne (TUTTE OBBLIGATORIE), nello stesso ordine:
{colonne_originali}
3. TUTTE le celle devono avere un valore (OBBLIGATORIAMENTE), NON POSSONO ESSERCI CELLE VUOTE

3. REGOLE CRITICHE:
- NON aggiungere colonne extra
- NON omettere nessuna colonna
- NON aggiungere testo o commenti
- NON utilizzare punti e virgola (;) come separatore, usa la virgola (,)
- TUTTE le celle devono avere un valore, non possono essere vuote o null
- Mantieni gli stessi tipi di dati per ogni colonna come nell'esempio
- Mantieni gli stessi formati dei valori come nell'esempio

RISPONDI ESCLUSIVAMENTE CON LE RIGHE CSV, NIENT'ALTRO
"""

payload = {
    "model": "mistral-small-2503",
    "messages": [
        {
            "role": "user",
            "content": prompt
        }
    ],
    "temperature": 0.1,
    "max_tokens": 12000,
}

response = requests.post(MISTRAL_ENDPOINT, headers=headers, json=payload)

if response.status_code == 200:
    result = response.json()['choices'][0]['message']['content']
    print(result)
    result = result.replace('\_', '_')

    csv_lines = [line for line in result.split('\n') if line.strip() and not line.startswith('*') and not line.startswith('#')]

    csv_content = '\n'.join(csv_lines)

    try:
        # Converti in DataFrame
        df = pd.read_csv(StringIO(csv_content))

        if not all(col in df.columns for col in df_esempio.columns):
            print("ERRORE: Colonne mancanti nella risposta")
            missing_cols = set(df_esempio.columns) - set(df.columns)
            print(f"Colonne mancanti: {missing_cols}")
        else:
            df = df[df_esempio.columns]

            # Salva il file
            output_path = 'ClassicPrompt/mistral-small-2503_synthetic_diabetes_data.csv'
            os.makedirs('ClassicPrompt', exist_ok=True)
            df.to_csv(output_path, index=False)
            print(f"File CSV salvato correttamente in: {output_path}")

    except Exception as e:
        print(f"Errore nel processing della risposta: {e}")
        print("Risposta ricevuta:")
        print(result)
else:
    print(f"Errore nella richiesta: {response.status_code}")
    print(response.text)

```csv
encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
123456,12345678,Caucasian,Female,[30-40),?,1,25,7,?,?,Pediatrics-Endocrinology,35,0,12,0,0,0,250.5,255,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,N